# UFRN Constructions's Finances
On this NoteBook we will present our discoveries over constructions and investments all over UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE ([UFRN](www.ufrn.br)).
All this made as a project for a subject on UFRN called `PROBABILIDADE`, where we choose some of several available open data about our university.

## Choosen Data
We decided to work over some financial values at UFRN, like how much money cost a specific work, how long it generally takes to finish a construction, the most expensive work, and so on. For that, we used the following 'csv' files:
- [obras.csv](http://dados.ufrn.br/dataset/a8b897f9-4659-44d4-842e-ac70ae21eb83/resource/067e7cad-934c-4134-a5d5-807915c074b4/download/obras.csv)
- [requisicaoobras.csv](http://dados.ufrn.br/dataset/15f55d0c-7ac8-4c1e-a73c-b0e37b9ef873/resource/0d923d18-1ffa-41d7-972f-33a12e9daf62/download/requisicaoobras.csv)

Importing some useful libs

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Just reading and showing the dataframe we are about to work with.

In [ ]:
works = pd.read_csv("res/obras.csv", sep=';')
work_requests = pd.read_csv("res/requisicaoobras.csv", sep=';', parse_dates=['data_envio'])

#### Works Data Frame

In [ ]:
print(works.info())
works.head()

#### Work Requests Data Frame

In [ ]:
print(work_requests.info())
work_requests.head()

## Obs.:
Unfortunately, the `data_envio` column, which is referent to when the request were submitted has null elements. Because of that, our inicial ideia of using this column as a `DateTimeIndex` has been destroyed.

### 1 ) Cleaning all unlabeled data
We choose to discharge those informations which had no unit responsible for the infrastructure work.
As a matter of fact, we have 473 infrastructure works with a responsible unit described (out of 846).

In [ ]:
valid_indexes = []
for idx, isnull in enumerate(works['unidade_responsavel'].isnull()):
    if (isnull == False ):
        valid_indexes.append(idx)

# Getting only valid data and re-index this new dataframe, since lots of datas would be excluded
works_filter = works.iloc[valid_indexes].reset_index(drop=True)
works_filter

### 1.1 )
Showing the number of requests attend per responsable unit.
Below, we can see that `Centro de Tecnologia` ( a tech center ) has executted a lot of demands.

In [ ]:
pd.DataFrame(works_filter['unidade_responsavel'].value_counts()).head()

### 1.2 )
Fixing string **'Valor'**

We will fix the value column to make it easier to handle on the data. It is important to remember that the works cost before was analyzed with two spaces after a comma being pennies.
Therefore:

Ex. ) `100` = ` RS 1,00` 

In [ ]:
#df = works_filter.copy()
df = works_filter.copy()
print(df.info())
pd.DataFrame(df[['valor', 'unidade_responsavel']]).head()


In [ ]:
for index, val in enumerate(df['valor']):
    val = val.replace(" ", "").replace("R$", "").replace(".","").replace(",", "")
    df['valor'][index] = val
df['valor'] = pd.to_numeric(df['valor'])

### 2 ) Money spent by department 

2.1 ) Coverting Strings to integers

In [ ]:
values = {}
dep_names = df['unidade_responsavel'].value_counts().index.tolist()
for i in dep_names:
    values[i] = 0
    
for index,row in df.iterrows():
    i = row['valor'].replace(" ", "")
    i = i.replace("R$", "")
    i = i.replace(".","")
    i = i.replace(",", "")
    i = int(i)
    values[row['unidade_responsavel']] += i

2.2 ) After converting strings to integers, now we can order the departments by the money spent.

In [ ]:
top11 = {}
# The 11 biggest spenders 
big_resources = sorted(values, key=values.get, reverse=True)[:11]
for i in big_resources:
    top11[i] = values[i]
          
campus_cerebro = list(top11.keys())[0]
CT = list(top11.keys())[1]
def without_keys(d, keys):
    return {k: v for k, v in d.items() if k not in keys}
# Excluding the 'campus do cerebro'
top10 = without_keys(top11,list(top11.keys())[0])

2.3 ) Now let's see the list with the top 11 departments with biggest expenses

In [ ]:
for names,values in top11.items():
    print (names ,":", values)

2.4 ) To aid our visualization of this result, let's generate a graphic.

In [ ]:
names = list(top11.keys())

fig, ax = plt.subplots(figsize=(15, 6))
bar_positions = np.arange(len(names)) + 0.75
xtick_pos = np.arange(len(names)) + 1
ax.set_yticks(xtick_pos)
ax.set_yticklabels(names)
ax.set_title('Value of the work by department')
ax.set_xlabel("Value")
ax.barh(bar_positions, list(top11.values()),0.75, color = 'orange', alpha = 0.7)
plt.show()

2.5 ) Oddly enough, the "Campus do Cérebro" costs a LOT more than the other departments,
the cell below shows the list of constructions made by that department

In [ ]:
cerebro_list = []
for idx,rows in df.iterrows():
    if (rows['unidade_responsavel'] == campus_cerebro):
        cerebro_list.append(idx)
        
df.loc[cerebro_list]


In [ ]:
cc_describe = df.loc[cerebro_list]
print(cc_describe.iloc[0]['descricao'])
print(cc_describe.iloc[1]['descricao'])

The cell bellow shows us that the huge expenses made by the "Campus do Cérebro" actually are from the construction of the campus itself, since this dataset dates only to 2005, at the time most of the central campus had already been made.

Oddly enough the "Instituto metrópole digital"(IMD) isn't on the dataset

2.6 ) Disparity in the analysis of values **without** the Campus do Cérebro (Brain Campus)

With this chart we can better analyze how the investments in each unit are established

In [ ]:
names = list(top10.keys())

fig, ax = plt.subplots(figsize=(15, 6))
bar_positions = np.arange(len(names)) + 0.75
xtick_pos = np.arange(len(names)) + 1
ax.set_yticks(xtick_pos)
ax.set_yticklabels(names)
ax.set_title('Value of the work by department without the CAMPUS DO CÉREBRO')
ax.set_xlabel("Value")
ax.barh(bar_positions, list(top10.values()),0.75 , color = 'green', alpha = 0.4)
plt.show()

## Analyzing Works Times
On the next cells, we will check, respectively, a construction's minimun time, max time and the mean of all these times.

### 3 )  The fastest work

3.1 ) This cell creates 2 variables, one containing the minimun value and the other it's index.

In [ ]:
works_min_time = works['qtd_dias'].min()
works_min_index_time = works['qtd_dias'].idxmin()

# Then, we print to observe it's values and costs.
print("The fastest work took",works_min_time,"days")
print("It also cost", works.iloc[works_min_index_time]['valor'])
pd.DataFrame(works.iloc[works_min_index_time])

### 4 ) The slowest work

4.1 ) This cell creates 2 variables, one containing the maximun value and the other it's index.

In [ ]:
works_max_time = works['qtd_dias'].max()
works_max_index_time = works['qtd_dias'].idxmax()

# Then, we print to observe it's values and costs.
print("The slowest work took",works_max_time,"days")
print("It also cost", works.iloc[works_max_index_time]['valor'])
pd.DataFrame(works.iloc[works_max_index_time])

In [ ]:
works_mean_time = works['qtd_dias'].mean()
print( int(works_mean_time),"is the mean amount of days taken to finish a construction on UFRN")

### 5 ) Considerations
After our analysis over times, we concluded that UFRN actually takes approximately 6 months for a construction to be finished.

In [ ]:
less_than_mean = 0
for i in works.qtd_dias:
    if i < works_mean_time:
        less_than_mean = less_than_mean + 1
        
amount_works = len(works.id_obra)
print(less_than_mean)
print(int (less_than_mean*100/amount_works),"%")

# Most of works done on UFRN are faster than the mean time.

### 6 ) Information on Requested Works

6.1 ) Following we will start our abstractions over the requested works of UFRN's departments.

For example, as we can see on the next cell, the department that made more requests is SuperIntendencia de Infraestrutura. 

In [ ]:
# Saving the 5 most requests values.
five_most_1 = work_requests['nome_unidade_requisitante'].value_counts()[0]
five_most_2 = work_requests['nome_unidade_requisitante'].value_counts()[1]
five_most_3 = work_requests['nome_unidade_requisitante'].value_counts()[2]
five_most_4 = work_requests['nome_unidade_requisitante'].value_counts()[3]
five_most_5 = work_requests['nome_unidade_requisitante'].value_counts()[4]

# SuperIntendencia de Infraestrutura has more than double requests than the second most requester(Residencias Universitárias).
pd.DataFrame(work_requests['nome_unidade_requisitante'].value_counts()[:10])


6.2 ) Difference between amount of requests  

To better visualize the difference between amount of requests, we shall plot a graphic on bars for compare. This bar plot shows the 5 departments that requested the most for works. It makes obvious that 'SuperIntendência de Infraestrutura' made lots of requests, specially if compared with the others from the top 5.

In [ ]:
# Create a subplot that returns both figure and axis.
fig, ax = plt.subplots(figsize=(15, 6))

# Imported function from numpy to set axis positions.
from numpy import arange
bar_positions = arange(5) + 1

# Define x_ticks, which represents each bar and it's titles.
tick_positions = range(1,6)
ax.set_title("Ten Most Requests for Department")
ax.set_yticks(tick_positions)
#ax.set_xticks("# Requests")
ax.set_xlabel("Requests")
#ax.set_ylabel(tick_positions)

x_names = ['SuperIntendência de Infraestrututra', 'Residencias Universitárias', 'UFRN', 'PROAD', 'Centro de Biociências']
ax.set_yticklabels(x_names)

# Give to bars the respectively amount of requests from each department.
num_cols = [five_most_1, five_most_2, five_most_3, five_most_4, five_most_5]
bar_heights = num_cols
ax.barh( bar_positions, bar_heights, 0.7, color = 'red', alpha = 0.7)
plt.show()

## Working with *Values*

In this part we will show in numbers and compare the values of the works with the nature of the requests.

### 7 ) Most expensive Work
The most expensive work was on Campus do Cérebro, researching about it we found that the Campus is located in Macaiba. According to the work description, the money was invested in the construction of the Lygia Maria Rocha Leão Laporta school and in the construction of the Search center. Because it is a project of an environment of academic studies focused on the neurological medical area it is justified the cost of about 76 million reais, The work took 2 years to finish. 

7.1 ) This cell shows in detail the two works carried out on the Campus do Cérebro.

In [ ]:
cerebro_list = []
CT_list = []
for idx,rows in df.iterrows():
    if (rows['unidade_responsavel'] == CT):
        CT_list.append(idx)
    elif (rows['unidade_responsavel'] == campus_cerebro):
        cerebro_list.append(idx)
        
df.loc[cerebro_list]

7.2 ) This cell shows a Picture of the Campus do Cérebro during the construction

In [ ]:
import tkinter as tk
root = tk.Tk()

imagem = tk.PhotoImage(file = "campus_cerebro.png")
w = tk.Label(root, image=imagem)
w.imagem = imagem
w.pack()

root.mainloop()

7.3 ) Here we use the data to find the most expensive done or in progress work

In [ ]:
most_exp_work = works['valor'].max()

this = 0

print("The most expensive work cost was:", most_exp_work)

for i,valor in enumerate(works['valor']):
    if(valor == most_exp_work):
        this = i    
        
print("Work: ", works['unidade_responsavel'][this])        
print("\nDescription: ", works['descricao'][this])

pd.DataFrame(works.iloc[this])

### 8 ) Cheaper Work
Strangely the cheapest work that appears in the data cost only R$ 1,00, which seems quite out of the standards found in the data frame. the work was requested by UFRN and as it is explicit in the work's description.

The value was designated to the construction of civil works of the Pilot Research Center of the International Institute of Neurosciences, name that is given to one of the main constructions of the Campus do Cérebro(Brain Campus), which does not seem to be a coincidence.

In [ ]:
most_cheap_work = works['valor'].min()

print("The most cheap work cost was:", most_cheap_work)

this = 0

for i,valor in enumerate(works['valor']):
    if(valor == most_cheap_work):
        this = i
        
print("Work: ", works['unidade_responsavel'][this])
print("\nDescription: ", works['descricao'][this])

pd.DataFrame(works.iloc[this])

### 9 ) Work requests per year

9.1 ) We can see that 2012 was a year of many applications, which almost lost to 2011. But what we can observe is that both annual ranges are within the data range of the Brain Campus as seen in topic 2.3.

In [ ]:
years = work_requests.copy()
yr = work_requests['ano'].value_counts()

print("Works per year:")
yea = pd.DataFrame(yr)
yea.head()

9.2 ) The chart can help us see the divergence in the number of requests for works over the years. Note the peaks in the years 2011 and 2012.

In [ ]:
work_requests.plot(kind='hist', y = 'ano', bins=30, figsize=(15,8), color = 'purple', alpha=0.3)
plt.xlabel('Years')
plt.title("Request per Year")
plt.show()

### 10 ) Work requests status 

10.1 ) We can see that most of the requests had their projects sent or were answered.

In [ ]:
status = work_requests.copy()
sta = work_requests['status'].value_counts()

print("The work requests number by status:")

stat = pd.DataFrame(sta)
soma_fim = stat[stat.index.str.contains('FINALIZADA')].sum()
soma_pend = stat[stat.index.str.contains('PENDENTE')].sum()
stat.loc['FINALIZADAS'] = [soma_fim['status']]
stat.loc['PENDENTE AUTORIZAÇÃO'] = [soma_pend['status']]
pure_stat = stat.drop(index = ['FINALIZADA_ATENDIMENTO','FINALIZADA'])
final_stat = pure_stat.drop(index = ['ENVIADA_LICITACAO','AUTORIZADA PELO CHEFE DA UNIDADE','CADASTRADA',
                            'ENVIADA','RETORNADA','ATENDIDA_LICITACAO','AUTORIZADA PARA INFRA',
                            'AUTORIZADA DIREÇÃO CENTRO','AUTORIZADA PARA LICITAÇÃO',
                            'PENDENTE AUTORIZAÇÃO CHEFE UNIDADE','PENDENTE AUTORIZAÇÃO CHEFIA'])

porce = final_stat/pure_stat.sum()*100
porce

10.2 ) We calculated the percentage of request statuses to better understand data statistics.

In [ ]:
explode = (0.1, 0.1, 0.1, 0.1, 0.3, 0.1, 0.1, 0.1, 0.3, 0.1)
labels = porce.index
fig = plt.figure(figsize=(15,8))
plt.pie(porce, explode=explode, autopct='%1.1f%%',shadow=True, startangle=30, radius=0.9)
plt.axis('equal')
plt.legend(loc='best',labels=labels, bbox_to_anchor=(-0.1,0.6))
plt.title("Work Requests Status")
plt.show()